In [8]:
import os
from google.cloud import bigquery as bq
from google.cloud import storage
from google.cloud import bigquery_storage
#from google.colab import auth
import csv
import pandas as pd
import string
import gcsfs

##DANGER! Turning on bc of regex warning in the csv loop
import warnings
warnings.filterwarnings('ignore')

In [9]:
bucket_name = 'dialogue_labels'

In [10]:
table_id = "citp-sm-reactions.reddit_clean_comments.subcategory_labeled"

storage_client = storage.Client()
bq_client = bq.Client()
bqstorageclient = bigquery_storage.BigQueryReadClient()

bucket = storage_client.get_bucket(bucket_name)

In [11]:
job_config = bq.LoadJobConfig(
    schema=[
        bq.SchemaField("post_id", bq.enums.SqlTypeNames.STRING),
        bq.SchemaField("link_id", bq.enums.SqlTypeNames.STRING),
        bq.SchemaField("sub_id", bq.enums.SqlTypeNames.STRING),
        bq.SchemaField("subreddit", bq.enums.SqlTypeNames.STRING),
        bq.SchemaField("unix_epoc_time", bq.enums.SqlTypeNames.INTEGER),
        bq.SchemaField("body", bq.enums.SqlTypeNames.STRING), 
        bq.SchemaField("csv_name", bq.enums.SqlTypeNames.STRING),
        bq.SchemaField("author", bq.enums.SqlTypeNames.STRING),
        
    ],
)

In [12]:
# last_csv = 'unpopularopinion_comments_20_labeled_sample.csv'

# for idx, blob in enumerate(bucket.list_blobs()):
#     if last_csv in blob.name:
#         print(idx)
#         break

In [13]:
##Run this if there are missing csvs in the table that need to be included

sql = """
    SELECT distinct(csv_name)
    FROM {}
""".format(table_id)
#print(sql)
# Start the query, passing in the extra configuration.
#query_job = bq_client.query(sql, job_config=job_config_sl)
df = (
    bq_client.query(sql)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
)
print(df.head())

                                            csv_name
0  predicted_march/PoliticalHumor/PoliticalHumor_...
1  predicted_march/PoliticalHumor/PoliticalHumor_...
2  predicted_march/GenderCritical/GenderCritical_...
3  predicted_march/Conservative/Conservative_comm...
4  predicted_march/Conservative/Conservative_comm...


In [ ]:
rem = string.punctuation
pattern = r"[{}]".format(rem)
failed_blobs = []



arg_cols = ['who_instead_of_what', 'we_vs_them',
              'fact_related_argument', 'structured_argument',
              'counter_argument_structure', 'emotional_language',
              'other', 'you_in_the_epicenter', 'empathy_reciprocity',
              'generalized_call', 'situational_call_for_action',
              'collective_rhetoric', 'ungrounded_argument']
cols = ['post_id', 'link_id', 'sub_id', 'subreddit', 'unix_epoc_time', 'body', 'nest_level'] + arg_cols + ['author', 'post_score']


failed_blobs = []
for idx, blob in enumerate(bucket.list_blobs()):
    
    if ("march" in blob.name) and (blob.name not in df['csv_name'].values) and ("old_politics" not in blob.name):   

        try:
            fs = gcsfs.GCSFileSystem(project='citp-sm-reactions')
            with fs.open('{b}/{f}'.format(b=bucket_name, f=blob.name)) as f:
                print(blob.name)
                df = pd.read_csv(f, low_memory=False, lineterminator='\n')

                df = df[["Post ID", "link id", "sub_id", "subreddit",  "Publish Date UTC", 'body', "nest_level",
                       "('who_instead_of_what',)", "('weVSthem',)",
                    "('Fact_related_argument',)", "('structured_argument',)",
                    "('counter_argument_structure',)", "('emotional_language',)",
                    "('other',)", "('you_in_the_epicenter',)", "('empathy_reciprocity',)",
                    "('generalised_call',)", "('situational_call_for_action',)",
                    "('collective_rhetoric',)", "('ungrounded_argument',)", "author", "Score"]]

                #Not sure if these columsn contain NA values. Adding this in as insurance for the subsequent dropna
                df[['Score']] = df[['Score']].fillna(value=-9999)
                df[['nest_level']] = df[['nest_level']].fillna(value=-9999)

                #hopefully this will get rid of the jagged rows
                df.dropna()
                df['body']=df['body'].str.replace(pattern, '')
                df['author']=df['author'].str.replace(pattern, '')
                df["nest_level"]=df["nest_level"].astype(float)
                df["Score"]=df["Score"].astype(float)
                df.columns = cols
                df["csv_name"]=blob.name

                if len(df) > 0:
                    job = bq_client.load_table_from_dataframe(df, table_id, job_config=job_config)  # Make an API request.
                    job.result() 
                    #print("got this far")

                    table = bq_client.get_table(table_id)  # Make an API request.
                    print(
                        "Loaded {} rows and {} columns to {}".format(
                            table.num_rows, len(table.schema), table_id
                        )
                    )
        except Exception as e:
            print(blob.name, e)
            failed_blobs.append(blob.name)

predicted_march/ukpolitics/ukpolitics_comments_512_labeled_sample.csv
Loaded 163711105 rows and 23 columns to citp-sm-reactions.reddit_clean_comments.subcategory_labeled
predicted_march/ukpolitics/ukpolitics_comments_513_labeled_sample.csv
Loaded 163721105 rows and 23 columns to citp-sm-reactions.reddit_clean_comments.subcategory_labeled
predicted_march/ukpolitics/ukpolitics_comments_514_labeled_sample.csv
Loaded 163731105 rows and 23 columns to citp-sm-reactions.reddit_clean_comments.subcategory_labeled
predicted_march/ukpolitics/ukpolitics_comments_515_labeled_sample.csv
Loaded 163741105 rows and 23 columns to citp-sm-reactions.reddit_clean_comments.subcategory_labeled
predicted_march/ukpolitics/ukpolitics_comments_516_labeled_sample.csv
Loaded 163751105 rows and 23 columns to citp-sm-reactions.reddit_clean_comments.subcategory_labeled
predicted_march/ukpolitics/ukpolitics_comments_517_labeled_sample.csv
Loaded 163761105 rows and 23 columns to citp-sm-reactions.reddit_clean_comments.